In [1]:
from read_checkpoint import get_state
from train_gpt2 import GPTConfig, GPT, write_model
from transformers import GPT2LMHeadModel, GPT2Config
import torch

In [ ]:
# config = GPTConfig(n_embd=16,n_layer=2,n_head=4,block_size=128,vocab_size=50257)
# model = GPT(config)

In [29]:
model = GPT.from_pretrained('gpt2')
model_hf = GPT2LMHeadModel.from_pretrained('gpt2')

loading weights from pretrained gpt: gpt2


/home/haris/miniconda3/envs/nlp/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [30]:
write_model(model, 'gpt2.bin', dtype='bfloat16')

padded vocab size from 50257 to 50304
wrote gpt2.bin


In [39]:
filename = 'gpt2.bin'
sd,config = get_state(filename, return_config=True)

In [40]:
hf_config = GPT2Config(
    vocab_size=config.vocab_size,
    n_positions=config.block_size,
    # n_ctx=config.block_size,
    n_embd=config.n_embd,
    n_layer=config.n_layer,
    n_head=config.n_head,
)
model_hf_sd = GPT2LMHeadModel(hf_config)

sd_hf = model_hf_sd.state_dict()

In [41]:
# copy while ensuring all of the parameters are aligned and match in names and shapes
sd_keys_hf = sd_hf.keys()
sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']

In [42]:
for k in sd_keys_hf:
    if any(k.endswith(w) for w in transposed):
        # special treatment for the Conv1D weights we need to transpose
        print(sd[k].shape)
        assert sd_hf[k].shape[::-1] == sd[k].shape
        with torch.no_grad():
            sd[k]=sd[k].t()
        print(sd[k].shape)
        print()

torch.Size([2304, 768])
torch.Size([768, 2304])

torch.Size([768, 768])
torch.Size([768, 768])

torch.Size([3072, 768])
torch.Size([768, 3072])

torch.Size([768, 3072])
torch.Size([3072, 768])

torch.Size([2304, 768])
torch.Size([768, 2304])

torch.Size([768, 768])
torch.Size([768, 768])

torch.Size([3072, 768])
torch.Size([768, 3072])

torch.Size([768, 3072])
torch.Size([3072, 768])

torch.Size([2304, 768])
torch.Size([768, 2304])

torch.Size([768, 768])
torch.Size([768, 768])

torch.Size([3072, 768])
torch.Size([768, 3072])

torch.Size([768, 3072])
torch.Size([3072, 768])

torch.Size([2304, 768])
torch.Size([768, 2304])

torch.Size([768, 768])
torch.Size([768, 768])

torch.Size([3072, 768])
torch.Size([768, 3072])

torch.Size([768, 3072])
torch.Size([3072, 768])

torch.Size([2304, 768])
torch.Size([768, 2304])

torch.Size([768, 768])
torch.Size([768, 768])

torch.Size([3072, 768])
torch.Size([768, 3072])

torch.Size([768, 3072])
torch.Size([3072, 768])

torch.Size([2304, 768])
torch.

In [43]:
m = model_hf_sd.load_state_dict(sd,strict=False)

In [44]:
m

_IncompatibleKeys(missing_keys=[], unexpected_keys=['transformer.h.0.attn.bias', 'transformer.h.1.attn.bias', 'transformer.h.2.attn.bias', 'transformer.h.3.attn.bias', 'transformer.h.4.attn.bias', 'transformer.h.5.attn.bias', 'transformer.h.6.attn.bias', 'transformer.h.7.attn.bias', 'transformer.h.8.attn.bias', 'transformer.h.9.attn.bias', 'transformer.h.10.attn.bias', 'transformer.h.11.attn.bias'])

In [45]:
m.unexpected_keys

['transformer.h.0.attn.bias',
 'transformer.h.1.attn.bias',
 'transformer.h.2.attn.bias',
 'transformer.h.3.attn.bias',
 'transformer.h.4.attn.bias',
 'transformer.h.5.attn.bias',
 'transformer.h.6.attn.bias',
 'transformer.h.7.attn.bias',
 'transformer.h.8.attn.bias',
 'transformer.h.9.attn.bias',
 'transformer.h.10.attn.bias',
 'transformer.h.11.attn.bias']

In [27]:
for k in m.unexpected_keys:
    assert k.endswith('.attn.masked_bias') or k.endswith('.attn.bias')

In [ ]:
model.load_state_dict(sd_hf)